In [10]:
# Dependency imports
import re
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline

# Notebook customizations
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

### Properties data prep

In [11]:
properties_df = pd.read_csv("./housingWebScraper/housingWebScraper/output/Property-lastrun.csv", sep="|", \
                            dtype={'zip_code': str}, encoding="latin1")
properties_df

,address,area_sqft,basement_type,built_year,city,exterior_type,heating_type,lot_size,no_of_baths,no_of_bedrooms,no_of_parking_spaces,no_of_stories,parking_type,property_type,property_url,state,tax_amount,tax_year,zip_code
0,160 162 Old Peckslip Rd,"1,352",NaN,1985.0,Holmes,NaN,NaN,0.74 acres,3.0,3.0,NaN,NaN,NaN,Multi-Family,https://www.trulia.com/p/ny/holmes/160-162-old-peckslip-rd-holmes-ny-12531--2349467107,NY,"$7,718.79",NaN,12531
1,10 Cliff Ct,"1,184",NaN,1997.0,Holmes,NaN,NaN,1.4 acres,2.5,3.0,NaN,NaN,NaN,Single-Family Home,https://www.trulia.com/p/ny/holmes/10-cliff-ct-holmes-ny-12531--2305202587,NY,"$8,320.40",2017,12531
2,26 Donovan Ln,"1,890",NaN,1992.0,Holmes,NaN,NaN,2.53 acres,2.5,3.0,NaN,NaN,NaN,Single-Family Home,https://www.trulia.com/p/ny/holmes/26-donovan-ln-holmes-ny-12531--2349447123,NY,"$9,260.24",2017,12531
3,3130 Grand Concourse #7R,"103,883",NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7r-bronx-ny-10458--2171936520,NY,NaN,2017,10468
4,3130 Grand Concourse #7S,"103,883",NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7s-bronx-ny-10458--2333495270,NY,NaN,2017,10458
5,3130 Grand Concourse #7P,"103,883",NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7p-bronx-ny-10458--2173798637,NY,NaN,2017,10458
6,3130 Grand Concourse #7N,"103,883",NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7n-bronx-ny-10458--2345496753,NY,NaN,2017,10458
7,3184 Grand Concourse #4E,"80,811",NaN,1965.0,Bronx,NaN,NaN,0.38 acres,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3184-grand-concourse-4e-bronx-ny-10458--2345502809,NY,NaN,2017,10458
8,4 Wheel Dr,NaN,NaN,NaN,Craryville,NaN,NaN,4.0 acres,NaN,NaN,NaN,NaN,NaN,Lot/Land,https://www.trulia.com/p/ny/craryville/4-wheel-dr-craryville-ny-12521--2204807965,NY,"$1,509.69",2017,12521
9,Winding Ln,NaN,NaN,NaN,Craryville,NaN,NaN,"7,405 sqft",NaN,NaN,NaN,NaN,NaN,Lot/Land,https://www.trulia.com/p/ny/craryville/winding-ln-craryville-ny-12521--2158446526,NY,$31.29,2017,12521


In [12]:
properties_df.tax_year.value_counts()

2017                        685378
3 Beds Price                44    
4 Beds Price                33    
3 Beds                      15    
2 Beds Price                14    
4 Beds                      13    
5 Beds Price                9     
6 Beds Price                5     
2 Beds                      5     
1 day on Trulia Price       5     
6 Beds                      5     
5 Beds                      4     
1 Bed Price                 4     
Built in 1920 Price         3     
7 Beds Price                3     
1 day on Trulia             3     
8 Beds Price                2     
10 Beds Price               2     
Built in 1932 Price         1     
Built in 1966 Price         1     
2 days on Trulia            1     
Built in 1957               1     
9 Beds Price                1     
525 days on Trulia Price    1     
7 Beds                      1     
Built in 1901 Price         1     
4 days on Trulia Price      1     
Built in 1975 Price         1     
1 Bath Price        

#### Need to update the crawler to account for these anomalies of these improperly parsed records 
#### It is actually not the crawler's fault since these records come from a different page design for which the crawler support isn't added yet,  but we need to ensure such pages get ignored in the first place

In [13]:
properties_df.loc[properties_df['address'].isnull()]

,address,area_sqft,basement_type,built_year,city,exterior_type,heating_type,lot_size,no_of_baths,no_of_bedrooms,no_of_parking_spaces,no_of_stories,parking_type,property_type,property_url,state,tax_amount,tax_year,zip_code
4585,NaN,"1,674",NaN,1955.0,NaN,NaN,NaN,1458 sqft,4.0,6.0,NaN,NaN,NaN,NaN,https://www.trulia.com/p/ny/bronx/2827-valentine-ave-bronx-ny-10458--68856,NaN,"1,674 sqft",6 Beds Price,NaN
8573,NaN,"1,825",NaN,1928.0,NaN,NaN,NaN,NaN,1.0,3.0,NaN,NaN,NaN,NaN,https://www.trulia.com/p/ny/new-rochelle/100-pelham-rd-1-b-new-rochelle-ny-10805--1001820853?rd=1,NaN,$148/sqft,3 Beds Price,NaN
8950,NaN,"2,276",NaN,1924.0,NaN,NaN,NaN,8420 sqft,2.0,3.0,NaN,NaN,NaN,NaN,https://www.trulia.com/p/ny/new-rochelle/38-leland-ave-new-rochelle-ny-10805--2009254748,NaN,"2,276 sqft",3 Beds Price,NaN
9365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.trulia.com/p/ny/new-rochelle/address-not-disclosed-new-rochelle-ny-10805--2009254071?rd=1,NaN,NaN,NaN,NaN
9661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.trulia.com/c/ny/new-rochelle/harbor-house-3-davenport-ave-new-rochelle-ny-10805--2123143985?rd=1,NaN,NaN,NaN,NaN
11535,NaN,"3,937",NaN,1955.0,NaN,NaN,NaN,0.40 acres,5.0,6.0,NaN,NaN,NaN,NaN,https://www.trulia.com/p/ny/bronxville/8-oakledge-rd-bronxville-ny-10708--2009226050,NaN,"3,937 sqft",6 Beds,NaN
11614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.trulia.com/p/ny/yonkers/128-winnebago-rd-yonkers-ny-10710--2009230033,NaN,NaN,NaN,NaN
14123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.trulia.com/p/ny/bronxville/230-pondfield-rd-bronxville-ny-10708--1103311026,NaN,NaN,NaN,NaN
15395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.trulia.com/p/ny/yonkers/address-not-disclosed-yonkers-ny-10704--2009207400?rd=1,NaN,NaN,NaN,NaN
17320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.trulia.com/p/ny/new-rochelle/17-horton-ave-new-rochelle-ny-10801--2009238629,NaN,NaN,NaN,NaN


#### Remove anomalous records

In [14]:
properties_df = properties_df[~properties_df['address'].isnull()]
properties_df.reset_index(drop=True, inplace=True)
properties_df

,address,area_sqft,basement_type,built_year,city,exterior_type,heating_type,lot_size,no_of_baths,no_of_bedrooms,no_of_parking_spaces,no_of_stories,parking_type,property_type,property_url,state,tax_amount,tax_year,zip_code
0,160 162 Old Peckslip Rd,"1,352",NaN,1985.0,Holmes,NaN,NaN,0.74 acres,3.0,3.0,NaN,NaN,NaN,Multi-Family,https://www.trulia.com/p/ny/holmes/160-162-old-peckslip-rd-holmes-ny-12531--2349467107,NY,"$7,718.79",NaN,12531
1,10 Cliff Ct,"1,184",NaN,1997.0,Holmes,NaN,NaN,1.4 acres,2.5,3.0,NaN,NaN,NaN,Single-Family Home,https://www.trulia.com/p/ny/holmes/10-cliff-ct-holmes-ny-12531--2305202587,NY,"$8,320.40",2017,12531
2,26 Donovan Ln,"1,890",NaN,1992.0,Holmes,NaN,NaN,2.53 acres,2.5,3.0,NaN,NaN,NaN,Single-Family Home,https://www.trulia.com/p/ny/holmes/26-donovan-ln-holmes-ny-12531--2349447123,NY,"$9,260.24",2017,12531
3,3130 Grand Concourse #7R,"103,883",NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7r-bronx-ny-10458--2171936520,NY,NaN,2017,10468
4,3130 Grand Concourse #7S,"103,883",NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7s-bronx-ny-10458--2333495270,NY,NaN,2017,10458
5,3130 Grand Concourse #7P,"103,883",NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7p-bronx-ny-10458--2173798637,NY,NaN,2017,10458
6,3130 Grand Concourse #7N,"103,883",NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7n-bronx-ny-10458--2345496753,NY,NaN,2017,10458
7,3184 Grand Concourse #4E,"80,811",NaN,1965.0,Bronx,NaN,NaN,0.38 acres,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3184-grand-concourse-4e-bronx-ny-10458--2345502809,NY,NaN,2017,10458
8,4 Wheel Dr,NaN,NaN,NaN,Craryville,NaN,NaN,4.0 acres,NaN,NaN,NaN,NaN,NaN,Lot/Land,https://www.trulia.com/p/ny/craryville/4-wheel-dr-craryville-ny-12521--2204807965,NY,"$1,509.69",2017,12521
9,Winding Ln,NaN,NaN,NaN,Craryville,NaN,NaN,"7,405 sqft",NaN,NaN,NaN,NaN,NaN,Lot/Land,https://www.trulia.com/p/ny/craryville/winding-ln-craryville-ny-12521--2158446526,NY,$31.29,2017,12521


In [15]:
properties_df.property_type.value_counts()

Single-Family Home           385186
Multi-Family                 130542
Lot/Land                     83665 
Coop                         24655 
Mobile/Manufactured          21203 
Farm/Ranch                   16046 
Unknown                      11809 
Condo                        10785 
Income/Investment            10108 
Townhouse                    9946  
Apartment                    39    
Apartment/Condo/Townhouse    33    
Name: property_type, dtype: int64

In [16]:
properties_df.state.value_counts()

NY    703989
Name: state, dtype: int64

#### Convert certain columns like area and currency from strings to number, and make them uniform

In [17]:
properties_df['area_sqft'] = properties_df['area_sqft'].str.replace(',', '').astype(float)
properties_df['tax_amount'] = properties_df['tax_amount'].str.replace(r'\$|,', '').astype(float)
properties_df

/Users/viveckh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/viveckh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,address,area_sqft,basement_type,built_year,city,exterior_type,heating_type,lot_size,no_of_baths,no_of_bedrooms,no_of_parking_spaces,no_of_stories,parking_type,property_type,property_url,state,tax_amount,tax_year,zip_code
0,160 162 Old Peckslip Rd,1352.0,NaN,1985.0,Holmes,NaN,NaN,0.74 acres,3.0,3.0,NaN,NaN,NaN,Multi-Family,https://www.trulia.com/p/ny/holmes/160-162-old-peckslip-rd-holmes-ny-12531--2349467107,NY,7718.79,NaN,12531
1,10 Cliff Ct,1184.0,NaN,1997.0,Holmes,NaN,NaN,1.4 acres,2.5,3.0,NaN,NaN,NaN,Single-Family Home,https://www.trulia.com/p/ny/holmes/10-cliff-ct-holmes-ny-12531--2305202587,NY,8320.40,2017,12531
2,26 Donovan Ln,1890.0,NaN,1992.0,Holmes,NaN,NaN,2.53 acres,2.5,3.0,NaN,NaN,NaN,Single-Family Home,https://www.trulia.com/p/ny/holmes/26-donovan-ln-holmes-ny-12531--2349447123,NY,9260.24,2017,12531
3,3130 Grand Concourse #7R,103883.0,NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7r-bronx-ny-10458--2171936520,NY,NaN,2017,10468
4,3130 Grand Concourse #7S,103883.0,NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7s-bronx-ny-10458--2333495270,NY,NaN,2017,10458
5,3130 Grand Concourse #7P,103883.0,NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7p-bronx-ny-10458--2173798637,NY,NaN,2017,10458
6,3130 Grand Concourse #7N,103883.0,NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7n-bronx-ny-10458--2345496753,NY,NaN,2017,10458
7,3184 Grand Concourse #4E,80811.0,NaN,1965.0,Bronx,NaN,NaN,0.38 acres,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3184-grand-concourse-4e-bronx-ny-10458--2345502809,NY,NaN,2017,10458
8,4 Wheel Dr,NaN,NaN,NaN,Craryville,NaN,NaN,4.0 acres,NaN,NaN,NaN,NaN,NaN,Lot/Land,https://www.trulia.com/p/ny/craryville/4-wheel-dr-craryville-ny-12521--2204807965,NY,1509.69,2017,12521
9,Winding Ln,NaN,NaN,NaN,Craryville,NaN,NaN,"7,405 sqft",NaN,NaN,NaN,NaN,NaN,Lot/Land,https://www.trulia.com/p/ny/craryville/winding-ln-craryville-ny-12521--2158446526,NY,31.29,2017,12521


In [18]:
unhandled_area_values = []
def make_area_uniform(row):
    '''
        Converts the areas in other units to sqft
    '''
    
    match = re.match(r"\s*(?P<area>[0-9,\.]+)\s+(?P<unit>[a-z]+)\s*$", str(row['lot_size']))
    if match is not None:
        area = match.group('area')
        unit = match.group('unit')
        area_in_float = float(area.replace(',', ''))
        if unit == 'acre' or unit == 'acres':
            row['lot_size'] = area_in_float * 43560
        elif unit == 'sqft':
            row['lot_size'] = area_in_float
        else:
            print("Unhandled unit for area - " + unit + ". Handle it in make_area_uniform function")
    else:
        if pd.notna(row['lot_size']):
            unhandled_area_values.append(str(row['lot_size']))
            print("Unhandled value in make_area_uniform " + str(row['lot_size']))
    
    return row

properties_df = properties_df.apply(lambda row: make_area_uniform(row), axis=1)

properties_df = properties_df[~properties_df['lot_size'].isin(unhandled_area_values)]
properties_df.reset_index(drop=True, inplace=True)
properties_df

Unhandled value in make_area_uniform 100x208
Unhandled value in make_area_uniform 89x130
Unhandled value in make_area_uniform 108x135
Unhandled value in make_area_uniform 74x140
Unhandled value in make_area_uniform 50x138
Unhandled value in make_area_uniform 69x119
Unhandled value in make_area_uniform 85x112
Unhandled value in make_area_uniform 130x159
Unhandled value in make_area_uniform 151x133
Unhandled value in make_area_uniform 66x135
Unhandled value in make_area_uniform 70x163
Unhandled value in make_area_uniform 70x125
Unhandled value in make_area_uniform 78x200
Unhandled value in make_area_uniform 65x151
Unhandled value in make_area_uniform 65x331
Unhandled value in make_area_uniform 85x200


,address,area_sqft,basement_type,built_year,city,exterior_type,heating_type,lot_size,no_of_baths,no_of_bedrooms,no_of_parking_spaces,no_of_stories,parking_type,property_type,property_url,state,tax_amount,tax_year,zip_code
0,160 162 Old Peckslip Rd,1352.0,NaN,1985.0,Holmes,NaN,NaN,32234.4,3.0,3.0,NaN,NaN,NaN,Multi-Family,https://www.trulia.com/p/ny/holmes/160-162-old-peckslip-rd-holmes-ny-12531--2349467107,NY,7718.79,NaN,12531
1,10 Cliff Ct,1184.0,NaN,1997.0,Holmes,NaN,NaN,60984,2.5,3.0,NaN,NaN,NaN,Single-Family Home,https://www.trulia.com/p/ny/holmes/10-cliff-ct-holmes-ny-12531--2305202587,NY,8320.40,2017,12531
2,26 Donovan Ln,1890.0,NaN,1992.0,Holmes,NaN,NaN,110207,2.5,3.0,NaN,NaN,NaN,Single-Family Home,https://www.trulia.com/p/ny/holmes/26-donovan-ln-holmes-ny-12531--2349447123,NY,9260.24,2017,12531
3,3130 Grand Concourse #7R,103883.0,NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7r-bronx-ny-10458--2171936520,NY,NaN,2017,10468
4,3130 Grand Concourse #7S,103883.0,NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7s-bronx-ny-10458--2333495270,NY,NaN,2017,10458
5,3130 Grand Concourse #7P,103883.0,NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7p-bronx-ny-10458--2173798637,NY,NaN,2017,10458
6,3130 Grand Concourse #7N,103883.0,NaN,1955.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3130-grand-concourse-7n-bronx-ny-10458--2345496753,NY,NaN,2017,10458
7,3184 Grand Concourse #4E,80811.0,NaN,1965.0,Bronx,NaN,NaN,16552.8,NaN,NaN,NaN,NaN,NaN,Coop,https://www.trulia.com/p/ny/bronx/3184-grand-concourse-4e-bronx-ny-10458--2345502809,NY,NaN,2017,10458
8,4 Wheel Dr,NaN,NaN,NaN,Craryville,NaN,NaN,174240,NaN,NaN,NaN,NaN,NaN,Lot/Land,https://www.trulia.com/p/ny/craryville/4-wheel-dr-craryville-ny-12521--2204807965,NY,1509.69,2017,12521
9,Winding Ln,NaN,NaN,NaN,Craryville,NaN,NaN,7405,NaN,NaN,NaN,NaN,NaN,Lot/Land,https://www.trulia.com/p/ny/craryville/winding-ln-craryville-ny-12521--2158446526,NY,31.29,2017,12521


In [19]:
properties_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704001 entries, 0 to 704000
Data columns (total 19 columns):
address                 704001 non-null object
area_sqft               548310 non-null float64
basement_type           0 non-null float64
built_year              541613 non-null float64
city                    703970 non-null object
exterior_type           0 non-null float64
heating_type            0 non-null float64
lot_size                678075 non-null object
no_of_baths             421586 non-null float64
no_of_bedrooms          409810 non-null float64
no_of_parking_spaces    0 non-null float64
no_of_stories           0 non-null float64
parking_type            0 non-null float64
property_type           704001 non-null object
property_url            704001 non-null object
state                   703973 non-null object
tax_amount              509535 non-null float64
tax_year                685374 non-null object
zip_code                703973 non-null object
dtypes: float64

In [20]:
# Convert the columns to their proper data types
properties_df['built_year'] = properties_df['built_year'].fillna(value=0)
properties_df['tax_year'] = properties_df['tax_year'].fillna(value=0)
properties_df = properties_df.astype(dtype={'area_sqft': np.float, 'built_year': np.int, 'lot_size': np.float, \
                                            'no_of_baths': np.float, 'no_of_bedrooms': np.float, \
                                            'tax_amount': np.float, 'tax_year': np.int})
properties_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704001 entries, 0 to 704000
Data columns (total 19 columns):
address                 704001 non-null object
area_sqft               548310 non-null float64
basement_type           0 non-null float64
built_year              704001 non-null int64
city                    703970 non-null object
exterior_type           0 non-null float64
heating_type            0 non-null float64
lot_size                678075 non-null float64
no_of_baths             421586 non-null float64
no_of_bedrooms          409810 non-null float64
no_of_parking_spaces    0 non-null float64
no_of_stories           0 non-null float64
parking_type            0 non-null float64
property_type           704001 non-null object
property_url            704001 non-null object
state                   703973 non-null object
tax_amount              509535 non-null float64
tax_year                704001 non-null int64
zip_code                703973 non-null object
dtypes: float64(1

### Transactions data prep

In [21]:
transactions_df = pd.read_csv("./housingWebScraper/housingWebScraper/output/Transaction-lastrun.csv", sep="|", \
                              encoding="latin1", parse_dates=['recording_date', 'contract_date'])
transactions_df

,contract_date,county_transfer_tax,document_type,price,property_url,recording_date,total_transfer_tax,transaction_type
0,1995-11-17,NaN,Deed,"$146,796",https://www.trulia.com/p/ny/holmes/160-162-old-peckslip-rd-holmes-ny-12531--2349467107,1995-12-12,NaN,Purchase/Resale Arm's Length Residential Transaction
1,2001-06-25,NaN,Deed,"$224,000",https://www.trulia.com/p/ny/holmes/10-cliff-ct-holmes-ny-12531--2305202587,2001-07-30,NaN,Purchase/Resale Arm's Length Residential Transaction
2,2013-09-19,NaN,Deed,"$70,000",https://www.trulia.com/p/ny/craryville/4-wheel-dr-craryville-ny-12521--2204807965,2013-09-19,NaN,Insured Non-Residential Grant Deed
3,2006-12-28,NaN,Deed,"$42,500",https://www.trulia.com/p/ny/craryville/4-wheel-dr-craryville-ny-12521--2204807965,2007-01-03,NaN,Insured Non-Residential Grant Deed
4,2018-11-08,NaN,Deed,"$325,000",https://www.trulia.com/p/ny/craryville/10-wang-dr-craryville-ny-12521--2011547166,2018-11-27,NaN,Purchase/Resale Arm's Length Residential Transaction
5,2016-08-12,NaN,Deed,"$122,000",https://www.trulia.com/p/ny/craryville/10-wang-dr-craryville-ny-12521--2011547166,2016-08-15,NaN,Purchase/Resale Arm's Length Residential Transaction
6,2004-07-21,NaN,Deed,"$130,000",https://www.trulia.com/p/ny/craryville/10-wang-dr-craryville-ny-12521--2011547166,2004-07-21,NaN,Purchase/Resale Arm's Length Residential Transaction
7,1998-08-27,NaN,Deed,"$385,000",https://www.trulia.com/p/ny/craryville/126-taghkanic-churchtown-rd-craryville-ny-12521--2349607986,1998-08-28,NaN,Non-Arm's Length Transaction
8,2009-04-17,NaN,Deed,"$437,500",https://www.trulia.com/p/ny/craryville/110-taghkanic-churchtown-rd-craryville-ny-12521--2011547172,2009-04-17,NaN,Purchase/Resale Arm's Length Residential Transaction
9,2004-03-08,NaN,Deed,"$363,750",https://www.trulia.com/p/ny/craryville/110-taghkanic-churchtown-rd-craryville-ny-12521--2011547172,2004-03-09,NaN,Insured Non-Residential Grant Deed


In [22]:
transactions_df.transaction_type.value_counts()

Purchase/Resale Arm's Length Residential Transaction    404640
Insured Non-Residential Grant Deed                      57760 
Non-Arm's Length Transaction                            29591 
REO and Trustee Deed                                    10935 
New Residential Construction Transaction                2137  
Name: transaction_type, dtype: int64

In [23]:
transactions_df.document_type.value_counts()

Deed                                                                                                                                                                                                                    292636
Bargain and Sale Deed                                                                                                                                                                                                   79755 
Warranty Deed                                                                                                                                                                                                           59797 
Other                                                                                                                                                                                                                   17144 
REO Resale                                                                                                  

#### Lets format the currency fields

In [24]:
transactions_df['price'] = transactions_df['price'].str.replace(r'\$|,', '').astype(float)
transactions_df['county_transfer_tax'] = transactions_df['county_transfer_tax'].str.replace(r'\$|,', '').astype(float)
transactions_df['total_transfer_tax'] = transactions_df['total_transfer_tax'].str.replace(r'\$|,', '').astype(float)
transactions_df

,contract_date,county_transfer_tax,document_type,price,property_url,recording_date,total_transfer_tax,transaction_type
0,1995-11-17,NaN,Deed,146796.0,https://www.trulia.com/p/ny/holmes/160-162-old-peckslip-rd-holmes-ny-12531--2349467107,1995-12-12,NaN,Purchase/Resale Arm's Length Residential Transaction
1,2001-06-25,NaN,Deed,224000.0,https://www.trulia.com/p/ny/holmes/10-cliff-ct-holmes-ny-12531--2305202587,2001-07-30,NaN,Purchase/Resale Arm's Length Residential Transaction
2,2013-09-19,NaN,Deed,70000.0,https://www.trulia.com/p/ny/craryville/4-wheel-dr-craryville-ny-12521--2204807965,2013-09-19,NaN,Insured Non-Residential Grant Deed
3,2006-12-28,NaN,Deed,42500.0,https://www.trulia.com/p/ny/craryville/4-wheel-dr-craryville-ny-12521--2204807965,2007-01-03,NaN,Insured Non-Residential Grant Deed
4,2018-11-08,NaN,Deed,325000.0,https://www.trulia.com/p/ny/craryville/10-wang-dr-craryville-ny-12521--2011547166,2018-11-27,NaN,Purchase/Resale Arm's Length Residential Transaction
5,2016-08-12,NaN,Deed,122000.0,https://www.trulia.com/p/ny/craryville/10-wang-dr-craryville-ny-12521--2011547166,2016-08-15,NaN,Purchase/Resale Arm's Length Residential Transaction
6,2004-07-21,NaN,Deed,130000.0,https://www.trulia.com/p/ny/craryville/10-wang-dr-craryville-ny-12521--2011547166,2004-07-21,NaN,Purchase/Resale Arm's Length Residential Transaction
7,1998-08-27,NaN,Deed,385000.0,https://www.trulia.com/p/ny/craryville/126-taghkanic-churchtown-rd-craryville-ny-12521--2349607986,1998-08-28,NaN,Non-Arm's Length Transaction
8,2009-04-17,NaN,Deed,437500.0,https://www.trulia.com/p/ny/craryville/110-taghkanic-churchtown-rd-craryville-ny-12521--2011547172,2009-04-17,NaN,Purchase/Resale Arm's Length Residential Transaction
9,2004-03-08,NaN,Deed,363750.0,https://www.trulia.com/p/ny/craryville/110-taghkanic-churchtown-rd-craryville-ny-12521--2011547172,2004-03-09,NaN,Insured Non-Residential Grant Deed


In [25]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521860 entries, 0 to 521859
Data columns (total 8 columns):
contract_date          503458 non-null datetime64[ns]
county_transfer_tax    82645 non-null float64
document_type          501235 non-null object
price                  521860 non-null float64
property_url           521860 non-null object
recording_date         521860 non-null datetime64[ns]
total_transfer_tax     170159 non-null float64
transaction_type       505063 non-null object
dtypes: datetime64[ns](2), float64(3), object(3)
memory usage: 31.9+ MB


### Writing to intermediate files to use for EDA

In [26]:
properties_df.to_csv("./output/engineered_trulia_properties.csv", sep="|", index=False, quoting=csv.QUOTE_NONE)

In [27]:
transactions_df.to_csv("./output/engineered_trulia_transactions.csv", sep="|", index=False, quoting=csv.QUOTE_NONE)